In [1]:
%load_ext tensorboard

In [2]:
from pandas import read_parquet
from data import file

data_train = read_parquet(file.news_articles_cleaned_train)
data_test = read_parquet(file.news_articles_cleaned_test)

In [3]:
data_train

,text_original,label,text_tokenized,text_tokenized_keywords,text_keywords,text_tokenized_lemmas,text_lemmas,text_tokenized_stemmed,text_stem
0,21-Jähriger fällt wohl bis Saisonende aus. Wie...,Sport,"[21-jähriger, fällt, wohl, bis, saisonende, au...","[21-jähriger, fällt, wohl, saisonende, ., wien...",21-jähriger fällt wohl saisonende . wien – rap...,"[21-jähriger, fällen, wohl, saisonende, wien, ...",21-jähriger fällen wohl saisonende wien rapid ...,"[21-jahrig, fallt, wohl, saison, wien, rapid, ...",21-jahrig fallt wohl saison wien rapid wohl sa...
1,"Erfundene Bilder zu Filmen, die als verloren g...",Kultur,"[erfundene, bilder, zu, filmen, ,, die, als, v...","[erfundene, bilder, filmen, ,, verloren, gelte...","erfundene bilder filmen , verloren gelten : ``...","[erfunden, bilder, filmen, verlieren, gelten, ...",erfunden bilder filmen verlieren gelten `` the...,"[erfund, bild, film, verlor, gelt, ``, the, fo...",erfund bild film verlor gelt `` the forbidd ro...
2,Der frischgekürte CEO Sundar Pichai setzt auf ...,Web,"[der, frischgekürte, ceo, sundar, pichai, setz...","[frischgekürte, ceo, sundar, pichai, setzt, um...",frischgekürte ceo sundar pichai setzt umgängli...,"[frischgekürte, ceo, sundar, pichai, setzen, u...",frischgekürte ceo sundar pichai setzen umgängl...,"[frischgekurt, ceo, sundar, pichai, setzt, umg...",frischgekurt ceo sundar pichai setzt umgang fu...
3,"Putin: ""Einigung, dass wir Menge auf Niveau vo...",Wirtschaft,"[putin, :, ``, einigung, ,, dass, wir, menge, ...","[putin, :, ``, einigung, ,, menge, niveau, jän...","putin : `` einigung , menge niveau jänner halt...","[putin, ``, einigung, menge, niveau, jänner, h...",putin `` einigung menge niveau jänner halten m...,"[putin, ``, einig, meng, niveau, jann, halt, '...",putin `` einig meng niveau jann halt '' moskau...
4,Estland sieht den künftigen österreichischen P...,Inland,"[estland, sieht, den, künftigen, österreichisc...","[estland, sieht, künftigen, österreichischen, ...",estland sieht künftigen österreichischen präsi...,"[estland, sehen, künftig, österreichisch, präs...",estland sehen künftig österreichisch präsident...,"[estland, sieht, kunftig, osterreich, prasiden...",estland sieht kunftig osterreich prasident est...
...,...,...,...,...,...,...,...,...,...
9240,Bernd Saurer war Bridge-Juniorenweltmeister un...,Inland,"[bernd, saurer, war, bridge-juniorenweltmeiste...","[bernd, saurer, bridge-juniorenweltmeister, ,,...","bernd saurer bridge-juniorenweltmeister , krau...","[bernd, sauer, bridge-juniorenweltmeister, kra...",bernd sauer bridge-juniorenweltmeister krauss ...,"[bernd, saur, bridge-juniorenweltmeist, krauss...",bernd saur bridge-juniorenweltmeist krauss sch...
9241,Sandhere soll in vergangener Woche bei Luftang...,International,"[sandhere, soll, in, vergangener, woche, bei, ...","[sandhere, vergangener, woche, luftangriff, ge...",sandhere vergangener woche luftangriff getötet...,"[sandhere, vergangen, woche, luftangriff, töte...",sandhere vergangen woche luftangriff töten wer...,"[sandh, vergang, woch, luftangriff, getotet, w...",sandh vergang woch luftangriff getotet word wa...
9242,Derzeit Konzeptgruppe in Berlin – Kein Komment...,Wirtschaft,"[derzeit, konzeptgruppe, in, berlin, –, kein, ...","[derzeit, konzeptgruppe, berlin, –, kommentar,...",derzeit konzeptgruppe berlin – kommentar apple...,"[derzeit, konzeptgruppe, berlin, kommentar, ap...",derzeit konzeptgruppe berlin kommentar apple m...,"[derzeit, konzeptgrupp, berlin, kommentar, app...",derzeit konzeptgrupp berlin kommentar appl mag...
9243,Landeshauptmann will den vierten Regierungssit...,Inland,"[landeshauptmann, will, den, vierten, regierun...","[landeshauptmann, vierten, regierungssitz, erh...",landeshauptmann vierten regierungssitz erhalte...,"[landeshauptmann, viert, regierungssitz, erhal...",landeshauptmann viert regierungssitz erhalten ...,"[landeshauptmann, viert, regierungssitz, erhal...",landeshauptmann viert regierungssitz erhalt fp...


In [4]:
X_train = data_train.text_tokenized_stemmed
y_train = data_train.label

X_test = data_test.text_tokenized_stemmed
y_test = data_test.label

In [9]:
from tensorflow.keras import Sequential
import tensorflow as tf
from tensorflow.keras.layers import Dense, Conv1D, GlobalMaxPooling1D, InputLayer, Bidirectional, LSTM
from preprocessing.text import extract_vocabulary
from models.text.layers import create_text_vectorization, create_word_embedding


def build_model(X_train, y_train, rnn_units = 32):
    
    vocabulary, embedding_length = extract_vocabulary(X_train, verbose=True)
    vectorize_layer = create_text_vectorization(vocabulary)
    embedding_layer = create_word_embedding(vocabulary)
    
    # return_sequences=True: required because of global max pooling
    bidirectional_lstm = Bidirectional(LSTM(rnn_units, return_sequences=True))
    global_max_pooling = GlobalMaxPool1D(name="global_max_pool")
    
    model = Sequential(name="rnn")
    model.add(InputLayer(input_shape=(1,), dtype=tf.string, name="text_input"))
    model.add(vectorize_layer)
    model.add(embedding_layer)
    model.add(bidirectional_lstm)
    model.add(global_max_pooling)
    model.add(Dense(output_classes, activation=tf.nn.softmax, name="prediction"))
    
    return model


model = build_model(X_train, y_train)
model.summary()

Median sequence length:       : 173
Percentil                     : 0.98)
Cutoff sequence length        : 589
Max sequence length           : 1699
Embedding length              : 589
Vocabulary length             : 162207


MemoryError: std::bad_alloc